## Install cobrapy

In [11]:
%pip install cobra --quiet

## Import C3 model from Github

In [20]:
# Retrieving the model
!wget -q https://raw.githubusercontent.com/sshameer/DielLeafModeling/refs/heads/main/C3_model.sbml


## Import functions required to read sbml, write sbml and perform pFBA

In [13]:

from cobra.io import read_sbml_model
from cobra.io import write_sbml_model
from cobra.flux_analysis import pfba

## Import the model and constrain diel leaf model for C3 photosynthesis

In [14]:
C3_model = read_sbml_model("C3_model.sbml")

In [15]:
# constraining GPT transport reaction
C3_model.reactions.get_by_id("G6P_Pi_pc1").lower_bound= 0
C3_model.reactions.get_by_id("G6P_Pi_pc1").upper_bound= 0
C3_model.reactions.get_by_id("G6P_Pi_pc2").lower_bound= 0
C3_model.reactions.get_by_id("G6P_Pi_pc2").upper_bound= 0

In [16]:
# constraining starch phosphorylation
C3_model.reactions.get_by_id("RXN_1826_p1").lower_bound= 0
C3_model.reactions.get_by_id("RXN_1826_p1").upper_bound= 0
C3_model.reactions.get_by_id("RXN_1826_p2").lower_bound= 0
C3_model.reactions.get_by_id("RXN_1826_p2").upper_bound= 0

In [17]:
# Setting photon uptake
PPFD = 500
C3_model.reactions.Photon_tx1.upper_bound = PPFD
C3_model.reactions.Photon_tx1.lower_bound = 0
# Setting maintenance cost
VATPase = 0.0049*PPFD+2.7851
C3_model.reactions.get_by_id("ATPase_tx1").bounds = (VATPase,VATPase)
C3sol = pfba(C3_model)

In [18]:
# constraining sucrose accumulation to model a starch storing leaf
C3_model.reactions.get_by_id("SUCROSE_v_dielTransfer").bounds = (0,0)

In [19]:
C3_model.summary()


Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e1,CO2_tx1,35.14,1,100.00%
WATER_e1,H2O_tx1,31.55,0,0.00%
NITRATE_e1,Nitrate_tx1,0.2171,0,0.00%
NITRATE_e2,Nitrate_tx2,0.1447,0,0.00%
OXYGEN_MOLECULE_e2,O2_tx2,2.023,0,0.00%
Photon_e1,Photon_tx1,500,0,0.00%
SULFATE_e1,SO4_tx1,0.002766,0,0.00%
PROTON_c1,unlProtHYPO_c1,0.3151,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e2,CO2_tx2,-1.846,1,100.00%


### From uptake fluxes we can see that the model reports an uptake of CO2, and photon during the day and O2 at night. The model also reports release of O2 during the day and CO2 at night, indicative of C3